In [77]:
pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 2.0 MB 1.6 MB/s eta 0:00:01     |█▉                              | 112 kB 1.4 MB/s eta 0:00:02
You should consider upgrading via the '/home/daiecasquero/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [111]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle, islice
import datetime as datetime
import glob
from matplotlib import pyplot
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, make_scorer
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
import warnings
import lightgbm as lgb

import category_encoders as ce
from sklearn.linear_model import BayesianRidge
from sklearn.cross_decomposition import PLSRegression

%matplotlib inline

plt.style.use('default')
sns.set(style="whitegrid")
pd.options.display.float_format = '{:20,.2f}'.format
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (15, 5)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from catboost import CatBoostRegressor

In [112]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler
# the model
from sklearn.ensemble import RandomForestClassifier
# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline
# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [113]:
train_labels = pd.read_csv('training_train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id')
train_values = pd.read_csv('training_train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [114]:
train = train_values.join(train_labels)
train.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3.00
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2.00
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3.00
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2.00
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3.00


In [115]:
train = train.drop('geo_level_1_id', axis=1)
train = train.drop('geo_level_2_id', axis=1)
train = train.drop('geo_level_3_id', axis=1)
train = train.drop('land_surface_condition', axis=1)
#train = train.drop('count_floors_pre_eq', axis=1)

In [116]:
#edad promedio segun cantidad de pisos
train['edad_promedio'] = train.groupby('count_floors_pre_eq')['age'].transform('mean')

In [117]:
#cantidad de tipos de techo segun edad
train['tipo_techo'] = train.groupby('age')['roof_type'].transform('count')
#train = train.drop('roof_type', axis=1)

In [118]:
#cantidad de familias por edificacion
train['flias_afectadas'] = train.groupby('building_id')['count_families'].transform('count')

In [119]:
train = train.drop('roof_type', axis=1)
train = train.drop('other_floor_type', axis=1)
train = train.drop('position', axis=1)
#train = train.drop('other_floor_type', axis=1)
train = train.drop('plan_configuration', axis=1)
train = train.drop('legal_ownership_status', axis=1)
train = train.drop('ground_floor_type', axis=1)
train = train.drop('foundation_type', axis=1)

In [134]:
train_value = train['damage_grade']
train_data = train.iloc[:, train.columns != 'age']

In [121]:
test_labels = pd.read_csv('test_train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id')
test_values = pd.read_csv('test_train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [122]:
test = test_values.join(test_labels,how="inner")

In [123]:
test = test.drop('geo_level_1_id', axis=1)
test = test.drop('geo_level_2_id', axis=1)
test = test.drop('geo_level_3_id', axis=1)
test = test.drop('land_surface_condition', axis=1)

In [124]:
#edad promedio segun cantidad de pisos
test['edad_promedio'] = test.groupby('count_floors_pre_eq')['age'].transform('mean')

In [125]:
#cantidad de tipos de techo segun edad
test['tipo_techo'] = test.groupby('age')['roof_type'].transform('count')

In [126]:
#cantidad de familias por edificacion
test['cantidad_flias'] = test.groupby('building_id')['count_families'].transform('count')

In [127]:
test = test.drop('roof_type', axis=1)
test = test.drop('other_floor_type', axis=1)
test = test.drop('position', axis=1)
#test = test.drop('other_floor_type', axis=1)
test = test.drop('plan_configuration', axis=1)
test = test.drop('legal_ownership_status', axis=1)
test = test.drop('ground_floor_type', axis=1)
test = test.drop('foundation_type', axis=1)


test = test.drop_duplicates('age')
#IDs = test[['age']].copy()
IDs = test[['damage_grade']].copy()
test = test.drop('age', axis=1)

In [135]:
test_value = test['damage_grade']
test_data = test.iloc[:, test.columns != 'building_id']

In [136]:
LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

In [137]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(train_data, train_value, train_size=0.7, random_state=1234)

In [101]:
modeloBR = BayesianRidge()
LogLoss = make_scorer(log_loss)

In [102]:
parameters = {'n_iter':[10, 50, 100, 300, 500, 1000], 'compute_score':[True, False], 'normalize':[True, False], 'copy_X':[True,False]}
gscv = GridSearchCV(modeloBR, parameters, cv=5, scoring=LogLoss)

In [139]:
train_values.dropna()
train_inf = np.isinf(train)
train[train_inf] = 0
train_values.replace([np.inf, -np.inf], np.nan, inplace=True)
train_values.fillna(train_values.median(), inplace=True)

train_data.dropna()
train_inf = np.isinf(train)
train_data[train_inf] = 0
train_data.replace([np.inf, -np.inf], np.nan, inplace=True)
train_data.fillna(train_data.median(), inplace=True)

In [140]:
x = np.linspace(0, 2*np.pi, 8)
y = np.sin(x) + np.random.normal(0, 0.4, 8)

In [141]:
x = x.reshape(-1, 1)

In [106]:
gscv.fit(train_data, train_values)

ValueError: y should be a 1d array, got an array of shape (167062, 38) instead.

In [ ]:
print(gscv.best_score_, gscv.best_estimator_)

In [100]:
modeloBR = BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=True, copy_X=True,
              fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=10,
              normalize=False, tol=0.001, verbose=False)
modeloBR.fit(train_data, train_value)
result = modeloBR.predict(test)

entregable_BR = IDs[['building_id']].copy()
entregable_BR['damage_grade'] = result

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
entregable_BR.to_csv('BR.csv', index=False)

In [142]:
lgb_train = lgb.Dataset(train_data, train_value)
lgb_eval = lgb.Dataset(train_data, train_value, reference=lgb_train)

params = {
    'objective': 'regression',
    'metric': 'logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
}

params = {
    'objective': 'regression',
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)

preds = gbm.predict(train_data, num_iteration=gbm.best_iteration)
logLoss = log_loss(train_value, preds)
print("logLoss: %f" % (logLoss))

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024489 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 202
[LightGBM] [Info] Number of data points in the train set: 167062, number of used features: 28
[LightGBM] [Info] Start training from score 1.790359
[1]	valid_0's l2: 0.996579
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 0.912214
[3]	valid_0's l2: 0.843851
[4]	valid_0's l2: 0.788462
[5]	valid_0's l2: 0.743546
[6]	valid_0's l2: 0.707131
[7]	valid_0's l2: 0.677603
[8]	valid_0's l2: 0.653634
[9]	valid_0's l2: 0.634212
[10]	valid_0's l2: 0.618433
[11]	valid_0's l2: 0.605623
[12]	valid_0's l2: 0.595222
[13]	valid_0's l2: 0.586769
[14]	valid_0's l2: 0.579896
[15]	valid_0's l2: 0.574307
[16]	valid_0's l2: 0.569755
[17]	valid_0's l2: 0.566043
[18]	valid_0's l2: 0.562946
[19]	valid_0's l2: 0.560467
[20]	va

[290]	valid_0's l2: 0.53498
[291]	valid_0's l2: 0.534899
[292]	valid_0's l2: 0.534866
[293]	valid_0's l2: 0.534837
[294]	valid_0's l2: 0.534783
[295]	valid_0's l2: 0.534738
[296]	valid_0's l2: 0.534695
[297]	valid_0's l2: 0.534652
[298]	valid_0's l2: 0.534624
[299]	valid_0's l2: 0.53461
[300]	valid_0's l2: 0.53458
[301]	valid_0's l2: 0.53453
[302]	valid_0's l2: 0.534499
[303]	valid_0's l2: 0.534439
[304]	valid_0's l2: 0.534394
[305]	valid_0's l2: 0.534358
[306]	valid_0's l2: 0.534318
[307]	valid_0's l2: 0.534275
[308]	valid_0's l2: 0.534252
[309]	valid_0's l2: 0.534213
[310]	valid_0's l2: 0.534181
[311]	valid_0's l2: 0.534144
[312]	valid_0's l2: 0.534122
[313]	valid_0's l2: 0.534085
[314]	valid_0's l2: 0.534068
[315]	valid_0's l2: 0.534056
[316]	valid_0's l2: 0.534024
[317]	valid_0's l2: 0.533989
[318]	valid_0's l2: 0.533935
[319]	valid_0's l2: 0.533883
[320]	valid_0's l2: 0.533853
[321]	valid_0's l2: 0.533804
[322]	valid_0's l2: 0.533762
[323]	valid_0's l2: 0.533748
[324]	valid_0's l2

[587]	valid_0's l2: 0.525109
[588]	valid_0's l2: 0.525077
[589]	valid_0's l2: 0.525047
[590]	valid_0's l2: 0.525004
[591]	valid_0's l2: 0.524986
[592]	valid_0's l2: 0.524951
[593]	valid_0's l2: 0.524923
[594]	valid_0's l2: 0.524909
[595]	valid_0's l2: 0.524891
[596]	valid_0's l2: 0.524864
[597]	valid_0's l2: 0.524837
[598]	valid_0's l2: 0.524803
[599]	valid_0's l2: 0.524778
[600]	valid_0's l2: 0.524758
[601]	valid_0's l2: 0.524723
[602]	valid_0's l2: 0.524687
[603]	valid_0's l2: 0.524668
[604]	valid_0's l2: 0.524636
[605]	valid_0's l2: 0.524607
[606]	valid_0's l2: 0.524585
[607]	valid_0's l2: 0.524567
[608]	valid_0's l2: 0.524531
[609]	valid_0's l2: 0.524501
[610]	valid_0's l2: 0.524467
[611]	valid_0's l2: 0.524444
[612]	valid_0's l2: 0.524429
[613]	valid_0's l2: 0.524383
[614]	valid_0's l2: 0.524353
[615]	valid_0's l2: 0.524343
[616]	valid_0's l2: 0.524315
[617]	valid_0's l2: 0.524278
[618]	valid_0's l2: 0.524266
[619]	valid_0's l2: 0.524258
[620]	valid_0's l2: 0.52424
[621]	valid_0's

[872]	valid_0's l2: 0.517737
[873]	valid_0's l2: 0.517725
[874]	valid_0's l2: 0.517702
[875]	valid_0's l2: 0.517685
[876]	valid_0's l2: 0.517671
[877]	valid_0's l2: 0.517647
[878]	valid_0's l2: 0.517624
[879]	valid_0's l2: 0.517591
[880]	valid_0's l2: 0.517575
[881]	valid_0's l2: 0.517552
[882]	valid_0's l2: 0.517523
[883]	valid_0's l2: 0.517483
[884]	valid_0's l2: 0.517455
[885]	valid_0's l2: 0.517419
[886]	valid_0's l2: 0.51739
[887]	valid_0's l2: 0.517373
[888]	valid_0's l2: 0.517341
[889]	valid_0's l2: 0.517336
[890]	valid_0's l2: 0.517308
[891]	valid_0's l2: 0.517281
[892]	valid_0's l2: 0.517255
[893]	valid_0's l2: 0.517233
[894]	valid_0's l2: 0.517213
[895]	valid_0's l2: 0.517185
[896]	valid_0's l2: 0.517164
[897]	valid_0's l2: 0.517138
[898]	valid_0's l2: 0.517107
[899]	valid_0's l2: 0.517091
[900]	valid_0's l2: 0.517054
[901]	valid_0's l2: 0.517024
[902]	valid_0's l2: 0.517008
[903]	valid_0's l2: 0.516989
[904]	valid_0's l2: 0.516968
[905]	valid_0's l2: 0.51695
[906]	valid_0's 

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [133]:
result = gbm.predict(test, num_iteration=gbm.best_iteration)

entregable_GBM = IDs[['building_id']].copy()
entregable_GBM['Target'] = result

KeyError: "None of [Index(['building_id'], dtype='object')] are in the [columns]"